In [227]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator


0it [00:00, ?it/s]


In [233]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [229]:
key = pd.read_csv('./key_trunc.csv')

In [230]:
sub = pd.read_csv("./sample_submission_1.csv")

In [231]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [194]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days)    

In [195]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days)    

In [196]:
def get_mean_diff(row, last_n_days):
    return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [197]:
def get_last_day_visits(row):
    return row[1:]

In [198]:
def get_long_stats(func_type, df, df_melt, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_median, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_mean, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean_diff":
        rolling_stats = df.iloc[:, -61:].progress_apply(get_mean_diff, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "last_day_visits":        
        rolling_stats = df.iloc[:, -61:].progress_apply(get_last_day_visits, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [199]:
def generate_train_melt():

#     train_melt = pd.read_csv("train_feats.csv")

#     Commented because these features are already generated
    train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

    train_melt = get_long_stats("mean", train, train_melt, 7)
    train_melt = get_long_stats("mean", train, train_melt, 30)
    train_melt = get_long_stats("mean", train, train_melt, 60)

    train_melt = get_long_stats("median", train, train_melt, 7)
    train_melt = get_long_stats("median", train, train_melt, 30)
    train_melt = get_long_stats("median", train, train_melt, 60)

    train_melt = get_long_stats("mean_diff", train, train_melt, 7)
    train_melt = get_long_stats("last_day_visits", train, train_melt)
    
    train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
    
    # Columns i am not using
    # train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
    # train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)
    
    return train_melt 

train_melt = generate_train_melt()
train_melt.to_csv("train_feats.csv", index = False)


  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=False).mean()
  

 57%|█████▋    | 82994/145063 [00:28<00:20, 3014.96it/s]


100%|█████████▉| 144901/145063 [00:50<00:00, 2824.25it/s]
145064it [00:53, 2717.63it/s]                            
  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=30,center=False).mean()
  

 13%|█▎        | 18961/145063 [00:06<00:40, 3135.22it/s]


 68%|██████▊   | 99179/145063 [00:34<00:16, 2787.06it/s]


100%|█████████▉| 144772/145063 [00:51<00:00, 2845.69it/s]
145064it [00:54, 2650.91it/s]                            
  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=60,center=False).mean()
  

 22%|██▏       | 31411/145063 [00:11<00:38, 2976.20it/s]


 76%|███████▌  | 110509/145063 [00:40<00:11, 3038.69it/s]


...

 76%|███████▋  | 6647725/8703780 [04:41<01:25, 23932.59it/s]


 84%|████████▎ | 7274803/8703780 [05:07<00:58, 24508.13it/s]


 91%|█████████ | 7928351/8703780 [05:33<00:28, 26978.85it/s]


 99%|█████████▉| 8632309/8703780 [05:59<00:02, 26651.59it/s]


100%|██████████| 8703780/8703780 [06:05<00:00, 23838.17it/s]

In [7]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [8]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [9]:
def get_month(row):
    return row.date.split("-")[1]

In [10]:
def get_date(row):
    return row.date.split("-")[2]

In [11]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [12]:
def get_median_7(row, last_index):
    
    return np.median(row.iloc[last_index - 7 : last_index].tolist())

In [13]:
def get_median_30(row, last_index):
    
    return np.median(row.iloc[last_index - 30 : last_index].tolist())

In [14]:
def get_median_60(row, last_index):
    
    return np.median(row.iloc[last_index - 60 : last_index].tolist())

In [15]:
def get_mean_7(row, last_index):
    
    return np.mean(row.iloc[last_index - 7 : last_index].tolist())

In [16]:
def get_mean_30(row, last_index):
    
    return np.mean(row.iloc[last_index - 30 : last_index].tolist())

In [17]:
def get_mean_60(row, last_index):
    
    return np.mean(row.iloc[last_index - 60 : last_index].tolist())

In [277]:
def get_mean_diff_test(row, last_index):
    return np.mean([int(x) for x in np.diff( row.iloc[last_index - 7 : last_index] ) ], dtype=None)

In [244]:
def get_last_day_visits_test(row, last_index):
    
    return row.iloc[last_index - 1]

In [18]:
# def get_test_feats(row, last_index):
    
#     subset_row = row.iloc[last_index - 60 : last_index].tolist()
    
#     median_7 = np.median(subset_row[-7:])
#     median_30 = np.median(subset_row[-30:])
#     median_60 = np.median(subset_row[-60:])

#     mean_7 = np.mean(subset_row[-7:])
#     mean_30 = np.mean(subset_row[-30:])
#     mean_60 = np.mean(subset_row[-60:])
    
#     return pd.Series([row.Page, median_7, median_30, median_60, mean_7, mean_30, mean_60])

In [66]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [219]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0

    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 200, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)
    
    return bst

In [205]:
label_train_melt = train_melt["visits"]

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

train_melt.date_num = pd.to_numeric(train_melt.date_num)

In [220]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)

bst = run_xgb(x_train, label_train, x_valid, label_valid)
# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:81056.5	validation-rmse:81141.7	train-smape:166.57	validation-smape:165.937
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:80072.2	validation-rmse:80865.6	train-smape:89.2449	validation-smape:89.903
[20]	train-rmse:79534.5	validation-rmse:80760.5	train-smape:65.3632	validation-smape:66.6032
[30]	train-rmse:79227.1	validation-rmse:80716.4	train-smape:53.4349	validation-smape:52.8563
[40]	train-rmse:79043.6	validation-rmse:80693.7	train-smape:43.9227	validation-smape:43.8825
[50]	train-rmse:78932.7	validation-rmse:80683.7	train-smape:40.9151	validation-smape:41.0287
[60]	train-rmse:78862.2	validation-rmse:80681	train-smape:40.5923	validation-smape:41.1602
Stopping. Best iteration:
[55]	train-rmse:78894.1	validation-rmse:80679.8	train-smape:40.517	validation-smape:40.5986



In [226]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

                feature    fscore
0               weekday  0.020903
1             median_30  0.049331
2               mean_30  0.056856
3              date_num  0.064381
4              median_7  0.067726
5                mean_7  0.160535
6           mean_diff_7  0.197324
7  last_day_visits_None  0.382943


In [32]:
def get_preds(test_feats):
    
    d_test = xgb.DMatrix(test_feats)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [237]:
def get_overall_df():
    
#     key = pd.read_csv("key_trunc.csv")
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [247]:
def test_and_pred(overall_df):
    
    pred_df = pd.DataFrame()
    
    for i in range(overall_df.columns.tolist().index('2017-01-01'), len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date

        last_index = overall_df.columns.tolist().index(last_date)

        test_feats = pd.DataFrame()
        
        test_feats["Page"] = overall_df.Page
        test_feats["median_7"] = overall_df.apply(get_median_7, axis = 1, last_index = last_index)
        test_feats["median_30"] = overall_df.apply(get_median_30, axis = 1, last_index = last_index)
        test_feats["median_60"] = overall_df.apply(get_median_60, axis = 1, last_index = last_index)
        test_feats["mean_7"] = overall_df.apply(get_mean_7, axis = 1, last_index = last_index)
        test_feats["mean_30"] = overall_df.apply(get_mean_30, axis = 1, last_index = last_index)
        test_feats["mean_60"] = overall_df.apply(get_mean_60, axis = 1, last_index = last_index)

        test_feats["mean_diff"] = overall_df.apply(get_mean_diff_test, axis = 1, last_index = last_index)
        test_feats["last_day_visits"] = overall_df.apply(get_last_day_visits_test, axis = 1, last_index = last_index)

#         test_feats["month"] = last_date.split("-")[1]
        test_feats["datenum"] = last_date.split("-")[2]
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()
        
        prediction = get_preds(test_feats)

        overall_df[last_date] = prediction
        pred_df = pd.concat([pred_df, pd.concat([overall_df.Page, prediction], axis = 1)], axis = 1)
        
    return overall_df

In [278]:
# overall_df = get_overall_df()
overall_df = test_and_pred(overall_df)
















  0%|          | 0/145063 [00:00<?, ?it/s]

2017-01-01

















  0%|          | 1/145063 [00:10<410:48:21, 10.19s/it]














  0%|          | 642/145063 [00:10<286:17:43,  7.14s/it]














  1%|          | 1291/145063 [00:10<199:30:28,  5.00s/it]














  1%|▏         | 1970/145063 [00:10<138:59:52,  3.50s/it]














  2%|▏         | 2591/145063 [00:10<96:52:41,  2.45s/it] 














  2%|▏         | 3218/145063 [00:10<67:31:05,  1.71s/it]














  3%|▎         | 3776/145063 [00:10<47:04:43,  1.20s/it]














  3%|▎         | 4431/145063 [00:10<32:48:15,  1.19it/s]














  4%|▎         | 5099/145063 [00:10<22:51:20,  1.70it/s]














  4%|▍         | 5764/145063 [00:11<15:55:28,  2.43it/s]














  4%|▍         | 6434/145063 [00:11<11:05:43,  3.47it/s]














  5%|▍         | 7118/145063 [00:11<7:43:48,  4.96it/s] 














  5%|▌         | 7816/145063 [00:11<5:23:07,  7.08it/s]














  6%|▌         | 8482/145063 [00:11<3:45:11, 10.11it/s]








 31%|███▏      | 45512/145063 [00:17<00:15, 6618.70it/s]














 32%|███▏      | 46175/145063 [00:17<00:14, 6607.98it/s]














 32%|███▏      | 46837/145063 [00:17<00:14, 6588.07it/s]














 33%|███▎      | 47500/145063 [00:17<00:14, 6599.43it/s]














 33%|███▎      | 48161/145063 [00:17<00:14, 6575.39it/s]














 34%|███▎      | 48848/145063 [00:17<00:14, 6660.08it/s]














 34%|███▍      | 49515/145063 [00:17<00:14, 6511.26it/s]














 35%|███▍      | 50168/145063 [00:18<00:14, 6481.26it/s]














 35%|███▌      | 50817/145063 [00:18<00:14, 6374.06it/s]














 35%|███▌      | 51471/145063 [00:18<00:14, 6421.13it/s]














 36%|███▌      | 52144/145063 [00:18<00:14, 6509.41it/s]














 36%|███▋      | 52796/145063 [00:18<00:14, 6473.30it/s]














 37%|███▋      | 53444/145063 [00:18<00:14, 6400.35it/s]














 37%|███▋      | 54085/145063 [00:18<00:14, 6306.81it/s]














 38%|█

 64%|██████▎   | 92266/145063 [00:24<00:07, 6726.91it/s]














 64%|██████▍   | 92950/145063 [00:24<00:07, 6759.84it/s]














 65%|██████▍   | 93630/145063 [00:24<00:07, 6610.18it/s]














 65%|██████▌   | 94295/145063 [00:24<00:07, 6488.62it/s]














 65%|██████▌   | 95004/145063 [00:25<00:07, 6656.77it/s]














 66%|██████▌   | 95673/145063 [00:25<00:07, 6541.18it/s]














 66%|██████▋   | 96402/145063 [00:25<00:07, 6746.65it/s]














 67%|██████▋   | 97081/145063 [00:25<00:07, 6712.12it/s]














 67%|██████▋   | 97755/145063 [00:25<00:07, 6603.44it/s]














 68%|██████▊   | 98446/145063 [00:25<00:06, 6690.01it/s]














 68%|██████▊   | 99117/145063 [00:25<00:06, 6678.74it/s]














 69%|██████▉   | 99787/145063 [00:25<00:06, 6541.38it/s]














 69%|██████▉   | 100443/145063 [00:25<00:06, 6449.65it/s]














 70%|██████▉   | 101090/145063 [00:25<00:06, 6448.89it/s]














 70%

 96%|█████████▌| 139433/145063 [00:31<00:00, 6595.05it/s]














 97%|█████████▋| 140094/145063 [00:31<00:00, 6412.04it/s]














 97%|█████████▋| 140738/145063 [00:32<00:00, 6312.17it/s]














 97%|█████████▋| 141400/145063 [00:32<00:00, 6398.81it/s]














 98%|█████████▊| 142042/145063 [00:32<00:00, 6335.73it/s]














 98%|█████████▊| 142677/145063 [00:32<00:00, 6231.37it/s]














 99%|█████████▉| 143302/145063 [00:32<00:00, 6220.84it/s]














 99%|█████████▉| 143925/145063 [00:32<00:00, 6112.05it/s]














100%|█████████▉| 144538/145063 [00:32<00:00, 5987.12it/s]














100%|██████████| 145063/145063 [00:34<00:00, 4257.50it/s]














  0%|          | 0/145063 [00:00<?, ?it/s]














  0%|          | 1/145063 [00:09<378:19:06,  9.39s/it]














  3%|▎         | 4642/145063 [00:09<256:21:02,  6.57s/it]














  7%|▋         | 9790/145063 [00:09<172:52:01,  4.60s/it]














 10%|▉    

NameError: global name 'get_preds' is not defined

In [240]:
overall_df.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-02-20,2017-02-21,2017-02-22,2017-02-23,2017-02-24,2017-02-25,2017-02-26,2017-02-27,2017-02-28,2017-03-01
0,2NE1_zh.wikipedia.org_all-access_spider,18,11,5,13,14,9,9,22,26,...,0,0,0,0,0,0,0,0,0,0
1,2PM_zh.wikipedia.org_all-access_spider,11,14,15,18,11,13,22,11,10,...,0,0,0,0,0,0,0,0,0,0
2,3C_zh.wikipedia.org_all-access_spider,1,0,1,1,0,4,0,3,4,...,0,0,0,0,0,0,0,0,0,0
3,4minute_zh.wikipedia.org_all-access_spider,35,13,10,94,4,26,14,9,11,...,0,0,0,0,0,0,0,0,0,0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [ ]:
# train_melt = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_melt = pd.merge(train_melt, key, left_on="Page", right_on="trunc_page", how="inner")
train_melt.drop("Page_x", inplace=True, axis = 1)
train_melt.drop("Page_y", inplace=True, axis = 1)

train_melt.to_csv("train_merged.csv", index = False)

In [93]:
train_merged_filtered = train_merged[["Id", "median_60_days"]]

In [111]:
sub = pd.merge(sub, train_merged_filtered, on = "Id")
sub.drop("Visits", inplace=True, axis = 1)
sub.columns = ["Id", "Visits"]

In [112]:
sub.to_csv("median_60_days.csv", index = False)

In [257]:
np.mean(np.array([1,2,3]))

2.0